In [1]:
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pandas_datareader.data as web
from matplotlib.finance import candlestick_ohlc
import matplotlib.dates as mdates
import bs4 as bs
import pickle
import requests
import mpld3
mpld3.enable_notebook()
%matplotlib inline

/home/harsh/anaconda3/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:106: MatplotlibDeprecationWarning: The finance module has been deprecated in mpl 2.0 and will be removed in mpl 2.2. Please use the module mpl_finance instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [2]:
df_JPM=pd.read_csv('JPM1.csv')

In [28]:
fast_factor=2/(12+1)
slow_factor=2/(26+1)
signal_factor=2/(9+1)
df=df_JPM
df['26_EMA_Close']=df_JPM['Close'].ewm(span=26,adjust=False).mean()
for index,row in df.iterrows():
    df['Fast_EMA']=(df['Close']-df['26_EMA_Close'])*fast_factor+df['26_EMA_Close']
    df['Slow_EMA']=(df['Close']-df['26_EMA_Close'])*slow_factor+df['26_EMA_Close']
    df['Difference']=df['Fast_EMA']-df['Slow_EMA']    
df['EMA_Difference']=df['Difference'].ewm(span=26,adjust=False).mean()
signal=[]
for index,row in df.iterrows():
    signal.append((df.iloc[index]['Difference']-df.iloc[index]['EMA_Difference'])*
                  signal_factor+df.iloc[index]['EMA_Difference'])
df_signal=pd.DataFrame(list(zip(signal)),columns=['Signal'])
df=pd.concat([df,df_signal],axis=1)
df['Histogram']=df['Difference']-df['Signal']
MACD=[]
for index,row in df[1:].iterrows():
    if df.iloc[index]['Histogram']>0 and df.iloc[index-1]['Histogram']<0:
        MACD.append(1)
    else:
        MACD.append(0)
df_MACD=pd.DataFrame(list(zip(MACD)),columns=['MACD'])
df=pd.concat([df,df_MACD],axis=1)

In [80]:
df['MACD'].value_counts()

0.0    1851
1.0     162
Name: MACD, dtype: int64

In [84]:
df.to_csv('JPM1.csv')